In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl

sys.path.append('../input/jsmp-exp001/')
from utils import *
from models import *

In [ ]:
CHECKPOINT = '../input/jsmp-exp001/EXP002-fold0.ckpt'
MEANS      = '../input/jsmp-exp001/f_mean.npy'
H_UNITS    = [126, 126, 126]
DROPOUT    = [.2, .2, .2, .2]
DEVICE     = torch.device("cuda:0")
TRESHOLD   = .5

In [ ]:
features     = [f'feature_{i}' for i in range(130)]
features_all = features +  ['cross_41_42_43', 'cross_1_2']
targets      = ['action', 'action_1', 'action_2', 'action_3', 'action_4']
f_mean       = np.load(MEANS)

In [ ]:
class JSMPModule(pl.LightningModule):
    def __init__(self, num_feats, n_classes, h_units=256, dropout=.2, lr=1e-3, wd=1e-5):
        super().__init__()
        self.save_hyperparameters()
        self.num_feats = num_feats
        self.n_classes = n_classes
        self.h_units   = h_units
        self.dropout   = dropout
        self.lr        = lr
        self.wd        = wd
        
        self.model     = MLP(num_feats, n_classes, h_units, dropout)
        
    def loss_fn(self, y_hat, y):
        return torch.functional.F.binary_cross_entropy_with_logits(y_hat, y, None)
        
    def forward(self, features):
        return self.model(features)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.model.parameters(), lr=self.lr, weight_decay=self.wd)
    
    def training_step(self, batch, batch_idx):
        x, y, extra_feats  = batch
        
        y_hat = self.model(x)
        loss  = self.loss(y_hat, y.type_as(y_hat))
        y_hat = y_hat.sigmoid()
        
        return loss
    

    def validation_step(self, batch, batch_idx):
        x, y, extra_feats  = batch
                        
        y_hat = self.model(x)
        loss  = self.loss(y_hat, y.type_as(y_hat))
        y_hat = y_hat.sigmoid()
        
        return metrics
            
    

In [ ]:
model = JSMPModule(num_feats=len(features_all), n_classes=len(targets), h_units=H_UNITS, dropout=DROPOUT)
model.load_from_checkpoint(CHECKPOINT)
model.cuda()
model.eval()

In [ ]:
import janestreet
# janestreet.competition.make_env.__called__ = False
env = janestreet.make_env()
iter_test = env.iter_test()

for (test_df, sample_prediction_df) in iter_test:
    
    if test_df['weight'].item() != 0:
        
        test_df['cross_41_42_43'] = test_df['feature_41'] + test_df['feature_42'] + test_df['feature_43']
        test_df['cross_1_2']      = test_df['feature_1'] / (test_df['feature_2'] + 1e-5)

        x = test_df[features_all].values[0]
        x = fillna_npwhere_njit(x, f_mean)
        x = torch.cuda.FloatTensor(x).unsqueeze(0)
        
        y_hat = model(x)[0].sigmoid()
        y_hat_action = torch.median(y_hat)
        y_hat_action = (y_hat_action >= TRESHOLD).detach().cpu().numpy()
        
        sample_prediction_df.action = int(y_hat_action)
        
    else: 
        sample_prediction_df.action = 0
        
    env.predict(sample_prediction_df)